In [5]:

import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf

In [6]:
board_data = pd.read_csv(
    'c4_game_database.csv/c4_game_database.csv'
)
X = board_data.iloc[:,:-1]
Y = board_data.iloc[:,-1]


exists = 2 in board_data
exists

False

In [8]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

dummy_y

X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33)
X_train

,pos_01,pos_02,pos_03,pos_04,pos_05,pos_06,pos_07,pos_08,pos_09,pos_10,pos_11,pos_12,pos_13,pos_14,pos_15,pos_16,pos_17,pos_18,pos_19,pos_20,pos_21,pos_22,pos_23,pos_24,pos_25,pos_26,pos_27,pos_28,pos_29,pos_30,pos_31,pos_32,pos_33,pos_34,pos_35,pos_36,pos_37,pos_38,pos_39,pos_40,pos_41,pos_42
308160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
23549,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,1.0
371675,0.0,-1.0,1.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,0.0,0.0,-1.0,1.0,1.0,1.0,-1.0,0.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0
126853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0,-1.0,-1.0,1.0,1.0,1.0,1.0,0.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0
318109,0.0,-1.0,1.0,1.0,-1.0,1.0,0.0,0.0,1.0,1.0,1.0,-1.0,1.0,0.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
108788,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,0.0,1.0,1.0,-1.0,-1.0,1.0,1.0,0.0
197946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,-1.0,1.0,0.0,1.0,-1.0,0.0,0.0,1.0,-1.0,0.0,-1.0,1.0,-1.0,0.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0
75232,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,0.0,0.0,0.0,-1.0,1.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,-1.0,1.0,-1.0,0.0,0.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.0


In [10]:
# create model
model = Sequential()
model.add(Dense(8, input_dim=42, activation='relu'))
model.add(Dense(4, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=15)


Epoch 1/15
7886/7886 [==============================] - 4s 501us/step - loss: 0.5477 - accuracy: 0.7740

In [13]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

model.save(r"C:\Users\Simon\Desktop\Projekte\Programmierpraktikum\Connect4\agents\agent_mcts_nn")

Test loss: 0.30429571866989136 / Test accuracy: 0.8745213150978088
INFO:tensorflow:Assets written to: C:\Users\Simon\Desktop\Projekte\Programmierpraktikum\Connect4\agents\agent_mcts_nn\assets


In [78]:
from agents.common import NO_PLAYER, BoardPiece, PLAYER2, PLAYER1, string_to_board
import numpy as np

initialBoard = np.ndarray(shape=(6, 7), dtype=BoardPiece)
initialBoard.fill(NO_PLAYER)

copy_board = initialBoard
#copy_board[5, 3] = PLAYER2
#copy_board[4, 3] = PLAYER2
#copy_board[3, 3] = PLAYER2
#copy_board[5, 2] = PLAYER1
#copy_board[4, 2] = PLAYER1
#copy_board[5, 4] = PLAYER1
#copy_board[4, 4] = PLAYER1
flat_board = copy_board.flatten()
input_board = pd.DataFrame(flat_board).T.replace(2, -1).astype(float)
print(input_board)

    0    1    2    3    4    5    6    7    8    9   ...   32   33   34   35  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

    36   37   38   39   40   41  
0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 42 columns]


In [79]:
# win player 2, draw, win player 1, ?
prediction = model.predict(input_board).round()
print(prediction)
#maximum = np.amax(prediction)
#print(maximum)
result = 1
if prediction[0, 0]==1:
    result = 2
elif prediction[0, 2]==1:
    result = 1
else:
    result = 0
    
print(result)


[[0. 0. 1. 0.]]
1


In [80]:
from keras.models import load_model

model.save('win_prediction_model.h5')  # creates a HDF5 file 'my_model.h5'

